In [ ]:
import pandas as pd
import seaborn as sns
from google.colab import drive

## Sprzedaż w piekarni The Bread Basket



W tym zeszycie zajmiemy się wizualizacją danych dotyczących sprzedaży w latach 2016-2017 w piekarni *The Bread Basket* w Edynburgu.

Dane pochodzą ze zbioru udostępnionego [na stronie kaggle](https://www.kaggle.com/akashdeepkuila/bakery) w ramach licencji *CC0*.

Zapoznajmy się z materiałem dostępnym w notebooku i na podstawie instrukcji przygotujmy dane, które będą potrzebne podczas zajęć.

**Uwaga.** Wracając do dokumentu, pamiętaj o ponownym wywołaniu komórek z kodem.

### Zawartość zbioru danych

W zaimportowanym niżej pliku **bakery_sales.csv** znajduje się 20 507 pozycji przypisanych do 9 684 transakcji klientów z informacjami o:


*   **TransactionNo** - numerze transakcji
*   **Items** - zakupionym produkcie
*   **DateTime** - czasie transakcji
*   **Daypart** - porze dnia (rano, po południu, wieczorem, w nocy)
*   **DayType** - typ dnia (dzień roboczy vs weekend).

Potrzebne nam dane zostały udostępnione wraz z dokumentem - poniższy kod importuje potrzebne pliki.

In [ ]:
drive.mount('/content/drive')

Dane importujemy do ramki danych pandas, dzięki czemu będziemy mogli sprawnie pracować nad ich wizualizacją.

In [ ]:
bakery_data = pd.read_csv('/content/drive/My Drive/Vis/Bakery Data/bakery_sales.csv')
bakery_data

## Przygotowanie zeszytu

Na początku chcemy upewnić się, że dane zostały poprawnie rozpoznane i dokonać potrzebnych konwersji.

Na podstawie poglądu danych powyżej spodziewamy się, że pierwsza kolumna zawiera informację w postaci kolejnych liczb całkowitych, w kolumnie drugiej znajdziemy nazwy różnych sprzedawanych produktów, kolumna trzecia powinna zawierać dane rozpoznane jako data z czasem, zaś ostatnie dwie kolumny powinny zawierać informacje w postaci tekstowej.

### Sprawdzenie typów danych

Poniżej potrzebujemy wykonać dołączone instrukcje, aby otrzymać ramki danych wykorzystywane na zajęciach.

Na początek sprawdźmy jak zostały rozpoznane dane podczas importu.

In [ ]:
bakery_data.dtypes

Upewnijmy się, czy w danych znajdują się rekordy, dla których brakuje informacji w którejkolwiek z kolumn.

In [ ]:
"complete records: " + str(len(bakery_data.dropna(how="any"))) + "; total records: " + str(len(bakery_data))

Przyjrzymy się jeszcze temu, jakie dane faktycznie ukrywają się pod typem **object** w przypadku każdej z kolumn.

In [ ]:
for column in bakery_data.columns:
  check_types = bakery_data[column].apply(lambda x: type(x))
  print(check_types.value_counts())

#### Konwersja daty

W przypadku czasu transakcji domyślnie jest ona identyfikowana jako *string*.

Zamieńmy dane w kolumnie **DateTime** na  *timestamp*.

In [ ]:
bakery_data["DateTime"] = pd.to_datetime(bakery_data["DateTime"])

Dodajmy nową kolumnę z datą transakcji o nazwie **Date** i sprawdźmy poprawność konwersji.

In [ ]:
bakery_data["Date"] = bakery_data["DateTime"].dt.date

In [ ]:
bakery_data["Date"].value_counts()

Ponieważ nie będziemy wykorzystywać informacji o czasie, pozostawmy w **bakery_data** jedynie kolumnę z datą.

In [ ]:
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Daypart", "DayType"]]
bakery_data

#### Przypisanie kategorii bazując na liczbie sprzedanych produktów

Przyjrzymy się bliżej zawartości kolumny **Items**.

In [ ]:
bakery_data["Items"].value_counts()

Widzimy, że w badanym okresie sprzedano wiele produktów z różną częstotliwością.

Wprowadzimy kategoryzację, dodając kolumnę **Item Categories**, która pozwoli nam wyróżnić 5 najpopularniejszych produktów, zaś pozostałym przypisać kategorię *Other*.

In [ ]:
product_categories = list(bakery_data["Items"].value_counts().index)[0:5]
product_categories.append("Other")
product_categories
bakery_data["Item Categories"] = pd.Series(pd.Categorical(bakery_data["Items"], categories=product_categories)).fillna("Other")
bakery_data

#### Konwersja pory dnia do kategorii

Przyjrzymy się bliżej zawartości kolumny **Daypart**.

In [ ]:
bakery_data["Daypart"].value_counts()

W przypadku tej kolumny lista kategorii jest krótka - zależy nam jedynie, by na wizualizacji kolejność pór dnia odzwierciedlała ich kolejność występowania.

Zdefiniujemy nową kolumnę **Day Part** i ustalimy właściwą kolejność kategorii i zastąpimy nią dotychczasową kolumnę **Daypart**.

In [ ]:
bakery_data["Day Part"] = pd.Series(pd.Categorical(bakery_data["Daypart"], categories=["Morning", "Afternoon", "Evening", "Night"]))
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Day Part", "DayType", "Item Categories"]]
bakery_data

#### Konwersja typu dnia do kategorii

Przyjrzyjmy się bliżej zawartości kolumny **Day Type**.

In [ ]:
bakery_data["DayType"].value_counts()

Podobnie jak w przypadku pory dnia, lista kategorii jest krótka - przygotujemy nową kolumnę **Day Type** w analogiczny sposób i usuniemy niepotrzebną kolumnę z zestawienia.

In [ ]:
bakery_data["Day Type"] = pd.Series(pd.Categorical(bakery_data["DayType"], categories=["Weekday", "Weekend"]))
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Day Part","Day Type", "Item Categories"]]
bakery_data

### Stworzenie ramek danych wykorzystywanych w wizualizacji

Poza samym zbiorem **bakery_data** na zajęciach będziemy potrzebowali jeszcze kilku perspektyw dla celów wizualizacji.

#### Statystyki dzienne

Poniżej liczymy ile produktów i w ramach ilu transacji kupowano dziennie z podziałem na typ dnia.

In [ ]:
items_daily = bakery_data[["Date","Day Type", "Items"]].groupby(["Date", "Day Type"]).count()
transactions_daily = bakery_data[["Date","Day Type", "TransactionNo"]].groupby(["Date", "Day Type"]).nunique()
daytype_statistics_daily = pd.merge(items_daily, transactions_daily, on=["Date", "Day Type"])
daytype_statistics_daily

Poniżej liczymy ile produktów i w ramach ilu transacji kupowano dziennie z podziałem na porę dnia.

In [ ]:
items_daily = bakery_data[["Date","Day Part", "Items"]].groupby(["Date", "Day Part"]).count()
transactions_daily = bakery_data[["Date","Day Part", "TransactionNo"]].groupby(["Date", "Day Part"]).nunique()
daypart_statistics_daily = pd.merge(items_daily, transactions_daily, on=["Date", "Day Part"])
daypart_statistics_daily

#### Statystyki kategorii

Na koniec zwracamy jeszcze liczbę produktów zakupionych w ramach transakcji z informacją o porze i typie dnia.

In [ ]:
items_count = bakery_data[["TransactionNo", "Items"]].groupby(["TransactionNo"]).count()
transactions_data = pd.merge(pd.DataFrame(bakery_data[["TransactionNo", "Day Type", "Day Part"]].drop_duplicates()), items_count, on="TransactionNo")
transactions_data

## Zadania